In [1]:
from sklearn.grid_search import GridSearchCV
import kagglegym
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from time import time
import itertools

C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
env = kagglegym.make()
o = env.reset()
#o.train = o.train[:1000]
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = [c for c in o.train.columns if c not in excl]



In [5]:
#train = pd.read_hdf(r'C:\Users\jiguo\Desktop\KProject\input\train.h5')
train = pd.read_hdf('../input/train.h5')
x_train = train[col]
y_train = train['y']
d_mean= x_train.median(axis=0)



In [6]:
#train = o.train[col]
n = x_train.isnull().sum(axis=1)
for c in x_train.columns:
    x_train[c + '_nan_'] = pd.isnull(x_train[c])
    d_mean[c + '_nan_'] = 0
x_train = x_train.fillna(d_mean)
x_train['znull'] = n
n = []



C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=3, random_state=17, verbose=0)
param_grid = { 
    'n_estimators': [50,75,100,150],
    'max_features': ['auto']#, 'sqrt', 'log2']
}

start=time()
CV_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 5)
CV_rfr.fit(x_train, y_train)
print CV_rfr.best_params_
end=time()
print(end-start)

{'max_features': 'auto', 'n_estimators': 75}
8748.18099999


In [8]:
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
param_grid = { 
    'n_estimators': [60,65,70,75,80,85],
    'max_features': ['auto']#, 'sqrt', 'log2']
}

start=time()
CV_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 5)
CV_rfr.fit(x_train, y_train)
print CV_rfr.best_params_
end=time()
print(end-start)

{'max_features': 'auto', 'n_estimators': 70}
10915.7940001


In [9]:
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=4, random_state=17, verbose=0)
param_grid = { 
    'n_estimators': [70,71,72,73,74],
    'max_features': ['auto']#, 'sqrt', 'log2']
}

start=time()
CV_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 5)
CV_rfr.fit(x_train, y_train)
print CV_rfr.best_params_
end=time()
print(end-start)

{'max_features': 'auto', 'n_estimators': 70}
11975.4519999


In [11]:
rfr = ExtraTreesRegressor(n_estimators=50, max_depth=4, n_jobs=4, random_state=17, verbose=0)
param_grid = { 
    'n_estimators': [45, 50],
    'max_features': ['sqrt', 'log2']
}

start=time()
CV_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 5)
CV_rfr.fit(x_train, y_train)
print CV_rfr.best_params_
end=time()
print(end-start)

{'max_features': 'sqrt', 'n_estimators': 50}
513.09100008


In [19]:
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
model1 = rfr.fit(x_train, y_train)
model1.feature_importances_




In [43]:
start=time()

while True:
    test = o.features[col]
    n = test.isnull().sum(axis=1)
    for c in test.columns:
        test[c + '_nan_'] = pd.isnull(test[c])
    test = test.fillna(d_mean)
    test['znull'] = n
    pred = o.target
    test2 = np.array\
    (o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
    pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * 0.65)+\
    (model2.predict(test2).clip(low_y_cut, high_y_cut) * 0.35)
    pred['y'] = pred.apply(lambda r: 0.95 * r['y'] + \
                           0.05 * ymean_dict[r['id']] \
                           if r['id'] in ymean_dict else r['y'], axis = 1)
    pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
    o, reward, done, info = env.step(pred)
    if done:
        print("el fin ...", info["public_score"])
        break
    if o.features.timestamp[0] % 100 == 0:
        print(reward)
        
end = timeit()
print(end - start)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-0.317732370655
-0.182276665545


KeyboardInterrupt: 